## 2021년 11월 10일 스터디
**Dacon 대회 - 따릉이 대여량 예측 경진대회**

2021/11/10 목주영

<span style="color:blue">**인사이트, 아쉬운 사항 제기 및 보완**</span>

<ol>
    <li>다른 특성들 추가해서 학습해보기</li>
    <li>다른 모델들로도 학습해보기</li>
    <li>K폴드 교차검증기법(K-fold Cross Validation) 사용하기</li>
</ol>

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

train = pd.read_csv("dataset/train.csv")
test = pd.read_csv('dataset/test.csv')
sample_submission = pd.read_csv('dataset/sample_submission.csv')
train.head()

,date_time,wind_direction,sky_condition,precipitation_form,wind_speed,humidity,low_temp,high_temp,Precipitation_Probability,number_of_rentals
0,2018-04-01,207.500,4.000,0.000,3.050,75.000,12.600,21.000,30.000,22994
1,2018-04-02,208.317,2.950,0.000,3.278,69.833,12.812,19.000,19.500,28139
2,2018-04-03,213.516,2.911,0.000,2.690,74.879,10.312,15.316,19.113,26817
3,2018-04-04,143.836,3.692,0.425,3.138,71.849,8.312,12.368,43.493,26034
4,2018-04-05,95.905,4.000,0.723,3.186,73.784,5.875,10.421,63.378,2833


In [2]:
train['date_time'] = pd.to_datetime(train['date_time'])
test['date_time'] = pd.to_datetime(test['date_time'])

train['day']=pd.DatetimeIndex(train['date_time']).day
test['day']=pd.DatetimeIndex(test['date_time']).day
train['month']=pd.DatetimeIndex(train['date_time']).month
test['month']=pd.DatetimeIndex(test['date_time']).month
train['year']=pd.DatetimeIndex(train['date_time']).year
test['year']=pd.DatetimeIndex(test['date_time']).year
train['weekday']=pd.DatetimeIndex(train['date_time']).weekday
test['weekday']=pd.DatetimeIndex(test['date_time']).weekday

In [3]:
train['day'][train['month'] == 5] += 30
train['day'][train['month'] == 6] += 61

test['day'][test['month'] == 5] += 30
test['day'][test['month'] == 6] += 61

<span style="color:blue">변수들을 차례대로 제거하며 학습해보자, 다른 모델들로도 학습해보자</span>

In [4]:
#1
# 변수 'date_time', 'number_of_rentals' 제거

In [5]:
train_label = train['number_of_rentals']
train.drop(['date_time', 'number_of_rentals'], axis = 1, inplace= True)
test.drop(['date_time'], axis = 1, inplace= True)

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   wind_direction             91 non-null     float64
 1   sky_condition              91 non-null     float64
 2   precipitation_form         91 non-null     float64
 3   wind_speed                 91 non-null     float64
 4   humidity                   91 non-null     float64
 5   low_temp                   91 non-null     float64
 6   high_temp                  91 non-null     float64
 7   Precipitation_Probability  91 non-null     float64
 8   day                        91 non-null     int64  
 9   month                      91 non-null     int64  
 10  year                       91 non-null     int64  
 11  weekday                    91 non-null     int64  
dtypes: float64(8), int64(4)
memory usage: 8.7 KB


In [7]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train,train_label)
sample_submission['number_of_rentals'] = rf.predict(test)
sample_submission.to_csv('bicycle_final_4.csv',encoding='UTF-8',index=False)

In [8]:
rf.score(train,train_label)

0.9812729193311751

In [9]:
# Xgboost 사용
from xgboost import XGBRegressor

xgb_reg = XGBRegressor()
xgb_reg.fit(train,train_label)
sample_submission['number_of_rentals'] = xgb_reg.predict(test)
sample_submission.to_csv('bicycle_xgb_reg_remove_only_date_time.csv',encoding='UTF-8',index=False)

In [10]:
print(xgb_reg.score(train,train_label))
best_score = 0
if (xgb_reg.score(train,train_label)>best_score):
    best_score = xgb_reg.score(train,train_label)
    print('update')

print("best score so far:",best_score)

0.9999999934154039
update
best score so far: 0.9999999934154039


<span style="color:blue">K폴드 교차검증기법(K-fold Cross Validation) 사용하기</span>

In [11]:
from sklearn.model_selection import KFold
kfolds = KFold(n_splits=10, shuffle = True, random_state=10)
scores = 0
n = 1
for idx_trn,idx_val in kfolds.split(train):
    #print("TRAIN:", idx_trn, "TEST:", idx_val)
    train_X, valid_X = train.iloc[idx_trn, :], train.iloc[idx_val, :]
    train_y, valid_y = train_label.iloc[idx_trn], train_label.iloc[idx_val]
    
    rf_reg = RandomForestRegressor(n_estimators = 200, max_depth = 8)
    rf_reg.fit(train_X, train_y) #모델 학습
    
    print("split %d:"%n, rf_reg.score(valid_X, valid_y))
    n+=1
    scores = scores + rf_reg.score(valid_X, valid_y)

print()
print("score:",scores/10)

split 1: 0.7226653946242914
split 2: 0.8860655890214741
split 3: 0.7718870111907871
split 4: 0.9097265863787438
split 5: 0.8241755384700378
split 6: 0.8224092617341587
split 7: 0.9412296245676668
split 8: 0.8918217265962385
split 9: 0.8852958826497019
split 10: 0.9068612293712447

score: 0.8562137844604344


In [12]:
#2
# 변수 'weekday' 제거

In [13]:
train.drop(['weekday'], axis = 1, inplace= True)
test.drop(['weekday'], axis = 1, inplace= True)

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   wind_direction             91 non-null     float64
 1   sky_condition              91 non-null     float64
 2   precipitation_form         91 non-null     float64
 3   wind_speed                 91 non-null     float64
 4   humidity                   91 non-null     float64
 5   low_temp                   91 non-null     float64
 6   high_temp                  91 non-null     float64
 7   Precipitation_Probability  91 non-null     float64
 8   day                        91 non-null     int64  
 9   month                      91 non-null     int64  
 10  year                       91 non-null     int64  
dtypes: float64(8), int64(3)
memory usage: 7.9 KB


In [15]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train,train_label)
sample_submission['number_of_rentals'] = rf.predict(test)
sample_submission.to_csv('bicycle_final_4.csv',encoding='UTF-8',index=False)

In [16]:
rf.score(train,train_label)

0.9809062130921339

In [17]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor()
xgb_reg.fit(train,train_label)
sample_submission['number_of_rentals'] = xgb_reg.predict(test)
sample_submission.to_csv('bicycle_xgb_reg.csv',encoding='UTF-8',index=False)

In [18]:
print(xgb_reg.score(train,train_label))

if (xgb_reg.score(train,train_label)>best_score):
    best_score = xgb_reg.score(train,train_label)
    print('update')

print("best score so far:",best_score)

0.9999999435594599
best score so far: 0.9999999934154039


In [19]:
from sklearn.model_selection import KFold
kfolds = KFold(n_splits=10, shuffle = True, random_state=10)
scores = 0
n = 1
for idx_trn,idx_val in kfolds.split(train):
    #print("TRAIN:", idx_trn, "TEST:", idx_val)
    train_X, valid_X = train.iloc[idx_trn, :], train.iloc[idx_val, :]
    train_y, valid_y = train_label.iloc[idx_trn], train_label.iloc[idx_val]
    
    rf_reg = RandomForestRegressor(n_estimators = 200, max_depth = 8)
    rf_reg.fit(train_X, train_y) #모델 학습
    
    print("split %d:"%n, rf_reg.score(valid_X, valid_y))
    n+=1
    scores = scores + rf_reg.score(valid_X, valid_y)

print()
print("score:",scores/10)

split 1: 0.7191935638181979
split 2: 0.8945178426751637
split 3: 0.7612699529724044
split 4: 0.9084206706544218
split 5: 0.8250784681493565
split 6: 0.8267036777613708
split 7: 0.9426448877425403
split 8: 0.8864140596100025
split 9: 0.8709626809943163
split 10: 0.9071843677740455

score: 0.8542390172151819


In [20]:
#3
# 변수 'wind_direction', 'Precipitation_Probability', 'month' 제거

In [21]:
# 원본
#train_label = train['number_of_rentals']
#train.drop(['date_time','wind_direction', 'Precipitation_Probability', 'month', 'number_of_rentals'], axis = 1, inplace= True)
#test.drop(['date_time','wind_direction', 'Precipitation_Probability', 'month'], axis = 1, inplace= True)

train.drop(['wind_direction', 'Precipitation_Probability', 'month'], axis = 1, inplace= True)
test.drop(['wind_direction', 'Precipitation_Probability', 'month'], axis = 1, inplace= True)

In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91 entries, 0 to 90
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sky_condition       91 non-null     float64
 1   precipitation_form  91 non-null     float64
 2   wind_speed          91 non-null     float64
 3   humidity            91 non-null     float64
 4   low_temp            91 non-null     float64
 5   high_temp           91 non-null     float64
 6   day                 91 non-null     int64  
 7   year                91 non-null     int64  
dtypes: float64(6), int64(2)
memory usage: 5.8 KB


In [23]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 0, n_estimators = 100)
rf.fit(train,train_label)
sample_submission['number_of_rentals'] = rf.predict(test)
sample_submission.to_csv('bicycle_final_4.csv',encoding='UTF-8',index=False)

In [24]:
rf.score(train,train_label)

0.9793031215137236

In [25]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor()
xgb_reg.fit(train,train_label)
sample_submission['number_of_rentals'] = xgb_reg.predict(test)
sample_submission.to_csv('bicycle_xgb_reg.csv',encoding='UTF-8',index=False)

In [26]:
print(xgb_reg.score(train,train_label))

if (xgb_reg.score(train,train_label)>best_score):
    best_score = xgb_reg.score(train,train_label)
    print('update')

print("best score so far:",best_score)

0.9999998530715722
best score so far: 0.9999999934154039


In [27]:
from sklearn.model_selection import KFold
kfolds = KFold(n_splits=10, shuffle = True, random_state=10)
scores = 0
n = 1
for idx_trn,idx_val in kfolds.split(train):
    #print("TRAIN:", idx_trn, "TEST:", idx_val)
    train_X, valid_X = train.iloc[idx_trn, :], train.iloc[idx_val, :]
    train_y, valid_y = train_label.iloc[idx_trn], train_label.iloc[idx_val]
    
    rf_reg = RandomForestRegressor(n_estimators = 200, max_depth = 8)
    rf_reg.fit(train_X, train_y) #모델 학습
    
    print("split %d:"%n, rf_reg.score(valid_X, valid_y))
    n+=1
    scores = scores + rf_reg.score(valid_X, valid_y)

print()
print("score:",scores/10)

split 1: 0.6689771484192915
split 2: 0.8981544732336267
split 3: 0.7908571578490365
split 4: 0.9313489766203605
split 5: 0.808571407124012
split 6: 0.8090654224429555
split 7: 0.9228438799366824
split 8: 0.8756738037259599
split 9: 0.869156466240964
split 10: 0.9126337895438013

score: 0.848728252513669


In [28]:
#import numpy as np

#def NMAE(true, pred):
#    score = np.mean(np.abs(true-pred) / true)
#    return score